# Using Google Colab with GitHub




[Google Colaboratory](http://colab.research.google.com) is designed to integrate cleanly with GitHub, allowing both loading notebooks from github and saving notebooks to github.

## Loading Public Notebooks Directly from GitHub

Colab can load public github notebooks directly, with no required authorization step.

For example, consider the notebook at this address: https://github.com/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb.

The direct colab link to this notebook is: https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb.

To generate such links in one click, you can use the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) Chrome extension.

## Browsing GitHub Repositories from Colab

Colab also supports special URLs that link directly to a GitHub browser for any user/organization, repository, or branch. For example:

- http://colab.research.google.com/github will give you a general github browser, where you can search for any github organization or username.
- http://colab.research.google.com/github/googlecolab/ will open the repository browser for the ``googlecolab`` organization. Replace ``googlecolab`` with any other github org or user to see their repositories.
- http://colab.research.google.com/github/googlecolab/colabtools/ will let you browse the main branch of the ``colabtools`` repository within the ``googlecolab`` organization. Substitute any user/org and repository to see its contents.
- http://colab.research.google.com/github/googlecolab/colabtools/blob/master will let you browse ``master`` branch of the ``colabtools`` repository within the ``googlecolab`` organization. (don't forget the ``blob`` here!) You can specify any valid branch for any valid repository.

## Loading Private Notebooks

Loading a notebook from a private GitHub repository is possible, but requires an additional step to allow Colab to access your files.
Do the following:

1. Navigate to http://colab.research.google.com/github.
2. Click the "Include Private Repos" checkbox.
3. In the popup window, sign-in to your Github account and authorize Colab to read the private files.
4. Your private repositories and notebooks will now be available via the github navigation pane.

## Saving Notebooks To GitHub or Drive

Any time you open a GitHub hosted notebook in Colab, it opens a new editable view of the notebook. You can run and modify the notebook without worrying about overwriting the source.

If you would like to save your changes from within Colab, you can use the File menu to save the modified notebook either to Google Drive or back to GitHub. Choose **File→Save a copy in Drive** or **File→Save a copy to GitHub** and follow the resulting prompts. To save a Colab notebook to GitHub requires giving Colab permission to push the commit to your repository.

## Open In Colab Badge

Anybody can open a copy of any github-hosted notebook within Colab. To make it easier to give people access to live views of GitHub-hosted notebooks,
colab provides a [shields.io](http://shields.io/)-style badge, which appears as follows:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

The markdown for the above badge is the following:

```markdown
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
```

The HTML equivalent is:

```HTML
<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
```

Remember to replace the notebook URL in this template with the notebook you want to link to.

In [1]:
!pip install Pyfhel

     |████████████████████████████████| 716 kB 22.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for Pyfhel: filename=Pyfhel-2.3.1-cp37-cp37m-linux_x86_64.whl size=6187022 sha256=3b4ee41f6796325c590a206c459f7ed8faf1668fcfa2bf060505b2f202a272ef
  Stored in directory: /root/.cache/pip/wheels/97/26/59/0bac9d03308c3f7b3bf5b39366949696d8ce34f4c2054206ff
Successfully built Pyfhel


In [3]:
"""
Client/Server demo with Pyfhel
========================================
Context Parameters shows how several parameters affect performance.
"""

from Pyfhel import Pyfhel, PyPtxt, PyCtxt
import tempfile
from pathlib import Path

# Using a temporary dir as a "secure channel"
# This can be changed into real communication using other python libraries.
secure_channel = tempfile.TemporaryDirectory()
sec_con = Path(secure_channel.name)
pk_file = sec_con / "mypk.pk"
contx_file = sec_con / "mycontx.con"


##### CLIENT
#HE Object Creation, including the public and private keys
HE = Pyfhel()
HE.contextGen(p=65537, m=2**12)
HE.keyGen() # Generates both a public and a private key

# Saving only the public key and the context
HE.savepublicKey(pk_file)
HE.saveContext(contx_file)

# Serializing two float values
a = 1.5
b = 2.5
ca = HE.encryptFrac(a)
cb = HE.encryptFrac(b)

ca.to_file(sec_con / "ca.ctxt")
cb.to_file(sec_con / "cb.ctxt")




##### SEMI-HONEST CLOUD
# Generating a second HE, acting as the honest-but-curious Cloud provider,
#  that will perform the operations and try to decrypt everything
HE_Cl = Pyfhel()
HE_Cl.restoreContext(contx_file)
HE_Cl.restorepublicKey(pk_file)

# loading the two ciphertexts. There is clearly potential for improvement here
c2a = PyCtxt(pyfhel=HE_Cl, fileName=sec_con / "ca.ctxt", encoding=float)
c2b = PyCtxt(pyfhel=HE_Cl, fileName=sec_con / "cb.ctxt", encoding=float)

# Attempting to decrypt results raises an error (missing secret key)
#> ---------------------------------------------------------------------------
#> RuntimeError                              Traceback (most recent call last)
#> Pyfhel/Pyfhel.pyx in Pyfhel.Pyfhel.Pyfhel.decryptFrac()
#> RuntimeError: Missing a Private Key [...]
try:
    print(HE_Cl.decrypt(c2a))
    raise Exception("This should not be reached!")
except RuntimeError:
    print("The cloud tried to decrypt, but couldn't!")

# The cloud operates with the ciphertexts:
c_mean = (c2a + c2b) / 2

# And sends the result back
c_mean.to_file(sec_con / "c_mean.ctxt")




##### CLIENT
# Load and decrypt Result
c_res = PyCtxt(pyfhel=HE, fileName=sec_con / "c_mean.ctxt", encoding=float)
print("Client decrypt results", c_res.decrypt())


# Cleaning up secure channel
secure_channel.cleanup()



# sphinx_gallery_thumbnail_path = 'static/thumbnails/clientServer.png'

The cloud tried to decrypt, but couldn't!
Client decrypt results 2.0
